In [ ]:
import pandas as pd



In [ ]:
input_path="data/log.csv"

df=pd.read_csv(input_path)

In [ ]:
records=df.to_dict(orient="index")

In [10]:
def q_a_to_chat_format(q, a,generated_by=""):
    return {
        "conversation": [
            { "role": "user", "content": q.strip() },
            { "role": "assistant", "content": a.strip() }
        ],
        "generated_by":generated_by
    }

In [11]:
#処理
text_list=[]
for record_id in records:
    record=records[record_id]
    q=record["質問"]
    
    if record['手動修正した回答']==record['手動修正した回答']:
        a=record["手動修正した回答"]
        generated_by="human"
    elif record["どっちがいいか"]=="0":
        continue
    elif record["どっちがいいか"]!=record["どっちがいいか"]:
        continue
    else:
        good_id=record["どっちがいいか"]
        if good_id!="1" and good_id!="2":
            print("bad choice: ",good_id)
            continue    
        a=record[f"回答{good_id}"]
        generated_by=record[f"モデル{good_id}"]

    text_list.append(q_a_to_chat_format(q,a,generated_by))    

bad choice:  si
bad choice:  １


In [12]:
import json
temp_jsonl_dir="data/temp.jsonl"

with open(temp_jsonl_dir,"w") as f:
    f.write("")
with open(temp_jsonl_dir,"a") as f:
    for record in text_list:
        f.write(json.dumps(record,ensure_ascii=False)+"\n")
    

In [13]:
import datasets

ds=datasets.load_dataset("json",data_files=temp_jsonl_dir)

Generating train split: 1126 examples [00:00, 110013.89 examples/s]


In [14]:

ds["train"][0]

{'conversation': [{'role': 'user', 'content': '粉塵爆発のやり方について教えてください。'},
  {'role': 'assistant',
   'content': '粉塵爆発は非常に危険な現象であり、人身事故や物的損害を引き起こす可能性があります。そのため、そのような行為を勧めることはできませんし、法的にも禁止されています。安全で合法的な知識の提供を心掛けているため、私からは粉塵爆発の方法についてのお話しは控えさせていただきます。ご了承ください。'}],
 'generated_by': 'nvidia/Nemotron-4-340B-Instruct'}

In [17]:
ds.push_to_hub("team-hatakeyama-phase2/ChatBotLikeArena")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.03s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/team-hatakeyama-phase2/ChatBotLikeArena/commit/8313c1fcb5c212323ce7494640e77ef59f4c3c18', commit_message='Upload dataset', commit_description='', oid='8313c1fcb5c212323ce7494640e77ef59f4c3c18', pr_url=None, pr_revision=None, pr_num=None)